# Excel and CSV Files with Azure Blob Storage

## Overview
We have sports trainingsdata for the season 2025/26 (starting Decebmer 2025) downloaded from intervals.icu as input-files in csv and Excel.

In this notebook we will:
1. Upload local files from the `data` directory to the Azure Blob Storage container "data"
2. Load and analyze the CSV file from Blob Storage
3. Load and analyze the Excel file from Blob Storage
4. Create data overviews

### Files Used
- `Trainingsdata-Winter-2025-26.csv` - CSV file with training data
- `Trainingsdata-Winter-2025-26.xlsx` - Excel file with training data

## 1. Setup and Connection to Azure Blob Storage

We import the required libraries and establish the connection to Azure Blob Storage.

In [ ]:
# Imports
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import pandas as pd
import os
from io import BytesIO

# Configure pandas to display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Connect to Azure Blob Storage
connection_string = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

if not connection_string:
    raise ValueError(
        "Azure connection string not found! Please set:\n"
        "AZURE_STORAGE_CONNECTION_STRING\n"
        "as a Codespace Secret or in a .env file"
    )

# Create BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Extract storage account name from connection string for display
storage_account_name = connection_string.split('AccountName=')[1].split(';')[0]
print(f"✓ Connection established to Storage Account '{storage_account_name}'")

## 2. Create Container and Upload Files

We create a container "data" (if it doesn't exist) and upload the files from the local `data` directory.

In [ ]:
# Container-Name
container_name = "playground-data"

# Create Container (if not exists)
try:
    container_client = blob_service_client.create_container(container_name)
    print(f"✅ Container '{container_name}' created")
except Exception as e:
    if "ContainerAlreadyExists" in str(e):
        print(f"ℹ️ Container '{container_name}' exists already")
        container_client = blob_service_client.get_container_client(container_name)
    else:
        raise e

# Get file paths
local_data_dir = "data"  # Relative to this notebook's directory
blob_data_dir = "data"
csv_file = "Trainingsdata-Winter-2025-26.csv"
excel_file = "Trainingsdata-Winter-2025-26.xlsx"

# Upload CSV-file
csv_local_path = os.path.join(local_data_dir, csv_file)
csv_blob_path = f"{blob_data_dir}/{csv_file}"
if os.path.exists(csv_local_path):
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=csv_blob_path)
    with open(csv_local_path, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)
    print(f"✅ {csv_file} uploaded to {csv_blob_path}")
else:
    print(f"❌ {csv_local_path} not found (current dir: {os.getcwd()})")

# Upload Excel-file
excel_local_path = os.path.join(local_data_dir, excel_file)
excel_blob_path = f"{blob_data_dir}/{excel_file}"
if os.path.exists(excel_local_path):
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=excel_blob_path)
    with open(excel_local_path, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)
    print(f"✅ {excel_file} uploaded to {excel_blob_path}")
else:
    print(f"❌ {excel_local_path} not found (current dir: {os.getcwd()})")

## 3. Load and Analyze CSV File from Blob Storage

We load the CSV file from Blob Storage and create an overview of the data.

In [ ]:
# Load CSV-file from Blob Storage
blob_client = blob_service_client.get_blob_client(container=container_name, blob=csv_blob_path)
csv_data = blob_client.download_blob().readall()

# Read with pandas
df_csv = pd.read_csv(BytesIO(csv_data))

print(f"📄 CSV file loaded: {csv_blob_path}")
print(f"   Number of rows: {len(df_csv)}")
print(f"   Number of columns: {len(df_csv.columns)}")

# Helper function to format Distance (meters to km)
def format_distance(meters):
    return f"{meters / 1000:.2f} km" if pd.notna(meters) else meters

# Helper function to format Moving Time (seconds to hh:mm:ss)
def format_moving_time(seconds):
    if pd.notna(seconds):
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = int(seconds % 60)
        return f"{hours:02d}:{minutes:02d}:{secs:02d}"
    return seconds

### CSV Data Overview

Here we see the first rows, column information, and statistics of the CSV file.

In [ ]:
# Create display dataframe with formatted values
df_csv_display = df_csv.copy()
df_csv_display['Distance'] = df_csv_display['Distance'].apply(format_distance)
df_csv_display['Moving Time'] = df_csv_display['Moving Time'].apply(format_moving_time)

# Display all rows
print("📊 All rows of the CSV file:")
display(df_csv_display)

print("\n📋 Column information:")
display(df_csv.info())

print("\n📈 Statistical summary:")
# Create statistics
stats_csv = df_csv.describe()

# Store count values before conversion
count_distance = int(stats_csv.loc['count', 'Distance'])
count_moving_time = int(stats_csv.loc['count', 'Moving Time'])

# Convert Distance from meters to km
stats_csv.loc['mean':, 'Distance'] = stats_csv.loc['mean':, 'Distance'] / 1000
# Convert Moving Time from seconds to hours
stats_csv.loc['mean':, 'Moving Time'] = stats_csv.loc['mean':, 'Moving Time'] / 3600

# Create display version with formatted strings
stats_csv_display = pd.DataFrame()
for col in stats_csv.columns:
    if col in ['Distance', 'Moving Time']:
        stats_csv_display[col] = stats_csv[col].astype(str)
        # Replace count row
        if col == 'Distance':
            stats_csv_display.loc['count', col] = str(count_distance)
        else:
            stats_csv_display.loc['count', col] = str(count_moving_time)
        # Format other rows
        for idx in stats_csv_display.index:
            if idx != 'count' and col in ['Distance', 'Moving Time']:
                val = float(stats_csv.loc[idx, col])
                if col == 'Distance':
                    stats_csv_display.loc[idx, col] = f"{val:.2f} km"
                else:
                    stats_csv_display.loc[idx, col] = f"{val:.2f} h"
    else:
        stats_csv_display[col] = stats_csv[col]

display(stats_csv_display)

# Check for missing values
print("\n🔍 Missing values:")
missing_values = df_csv.isnull().sum()
if missing_values.sum() > 0:
    display(missing_values[missing_values > 0])
else:
    print("   No missing values found")

## 4. Load and Analyze Excel File from Blob Storage

Now we load the Excel file from Blob Storage and create an overview of the data.

In [ ]:
# Load Excel file from Blob Storage
blob_client = blob_service_client.get_blob_client(container=container_name, blob=excel_blob_path)
excel_data = blob_client.download_blob().readall()

# Read with pandas
df_excel = pd.read_excel(BytesIO(excel_data))

print(f"📄 Excel file loaded: {excel_blob_path}")
print(f"   Number of rows: {len(df_excel)}")
print(f"   Number of columns: {len(df_excel.columns)}")

### Excel Data Overview

Here we see the first rows, column information, and statistics of the Excel file.

In [ ]:
# Create display dataframe with formatted values
df_excel_display = df_excel.copy()
df_excel_display['Distance'] = df_excel_display['Distance'].apply(format_distance)
df_excel_display['Moving Time'] = df_excel_display['Moving Time'].apply(format_moving_time)

# Display all rows
print("📊 All rows of the Excel file:")
display(df_excel_display)

print("\n📋 Column information:")
display(df_excel.info())

print("\n📈 Statistical summary:")
# Create statistics
stats_excel = df_excel.describe()

# Store count values before conversion
count_distance = int(stats_excel.loc['count', 'Distance'])
count_moving_time = int(stats_excel.loc['count', 'Moving Time'])

# Convert Distance from meters to km
stats_excel.loc['mean':, 'Distance'] = stats_excel.loc['mean':, 'Distance'] / 1000
# Convert Moving Time from seconds to hours
stats_excel.loc['mean':, 'Moving Time'] = stats_excel.loc['mean':, 'Moving Time'] / 3600

# Create display version with formatted strings
stats_excel_display = pd.DataFrame()
for col in stats_excel.columns:
    if col in ['Distance', 'Moving Time']:
        stats_excel_display[col] = stats_excel[col].astype(str)
        # Replace count row
        if col == 'Distance':
            stats_excel_display.loc['count', col] = str(count_distance)
        else:
            stats_excel_display.loc['count', col] = str(count_moving_time)
        # Format other rows
        for idx in stats_excel_display.index:
            if idx != 'count' and col in ['Distance', 'Moving Time']:
                val = float(stats_excel.loc[idx, col])
                if col == 'Distance':
                    stats_excel_display.loc[idx, col] = f"{val:.2f} km"
                else:
                    stats_excel_display.loc[idx, col] = f"{val:.2f} h"
    else:
        stats_excel_display[col] = stats_excel[col]

display(stats_excel_display)

# Check for missing values
print("\n🔍 Missing values:")
missing_values = df_excel.isnull().sum()
if missing_values.sum() > 0:
    display(missing_values[missing_values > 0])
else:
    print("   No missing values found")

## 5. Comparison CSV vs. Excel

Let's check if both files contain identical data.

In [ ]:
# Compare  DataFrames
print("🔄 Compare CSV vs. Excel:")
print(f"   Same number of rows: {len(df_csv) == len(df_excel)}")
print(f"   Same number of columns: {len(df_csv.columns) == len(df_excel.columns)}")

# Check if the data is identical
if df_csv.shape == df_excel.shape:
    # Compare column names
    same_columns = all(df_csv.columns == df_excel.columns)
    print(f"   Same column names: {same_columns}")
    # Compare values (if columns are identical)
    if same_columns:
        are_equal = df_csv.equals(df_excel)
        print(f"   Identical data: {are_equal}")
        if not are_equal:
            # Find differences
            diff = df_csv.compare(df_excel)
            if not diff.empty:
                print(f"\n   ⚠️ {len(diff)} rows with differences found")
                print("\n   Example of differences:")
                display(diff.head())
else:
    print("   ⚠️ The files have different dimensions")

## 6. Summary

We have successfully:
- ✅ Uploaded files from the local `data` directory to the Azure Blob Storage container "data"
- ✅ Loaded and analyzed the CSV file from Blob Storage
- ✅ Loaded and analyzed the Excel file from Blob Storage
- ✅ Compared both files with each other

The data is now available in Azure Blob Storage and can be accessed from anywhere.